# Mapping CoBinding matrix and SE matrix

In [2]:
import os
import pandas as pd
import sys
sys.path.insert(0, '../..')
import itertools
from scipy import stats

import numpy as np
from natsort import natsorted, ns

from genepy.epigenetics import chipseq as chip
from genepy.utils import helper
from genepy.utils import plot
from genepy.google import gcp
import igv
import SimpSOM as sps
from scipy import stats

import seaborn as sns
from matplotlib import cm
from matplotlib import pyplot as plt
from IPython.display import IFrame
import seaborn as sns
from bokeh.plotting import *
import igv

import numba
from numba import jit

from scipy.cluster.hierarchy import linkage, leaves_list
from sklearn.cluster import AgglomerativeClustering, DBSCAN, KMeans, OPTICS
from sklearn.mixture import GaussianMixture
from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from IPython.display import IFrame

from pybedtools import BedTool
import pyBigWig

output_notebook()
%load_ext autoreload
%autoreload 2

Loading BokehJS ...

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Cobinding mapping

In [5]:
project="Cobinding_SE"

cobinding_version="v3_remove_single_150"
se_matrix="SE.6868.xlsx"

In [6]:
cobinding_project="Cobinding_ChIP"
cobinding_matrix="merged.bed.gz"

results > Cobinding_ChIP > v3_remove_single_150 > merged.bed.gz

In [7]:
# open cobinding matrix
bindings= pd.read_csv('../results/'+cobinding_project+'/'+cobinding_version+'/'+cobinding_matrix, 
                      sep='\t', index_col=None, header=0,
                      low_memory=False)

In [8]:
bindings.head()

,chrom,start,end,peak_number,foldchange,-log10pvalue,-log10qvalue,relative_summit_pos,CEBPA,HOXA9,...,TFAP4,RUNX2,MAX,IRF8,MYB,ZEB2,RXRA,SPI1,MEF2C,FLAG_MEF2D
0,chr1,9936,10300,0,10.264422,31242.919925,20942.926956,107,0.0,0.0,...,20.13070,5.750405,0.0,0.0,0.0,0.0,8.51315,0.0,0.0,3.86739
1,chr1,15524,15801,1,3.326400,9.112810,7.086980,277,0.0,0.0,...,0.00000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000
2,chr1,29182,29649,2,15.734000,43.992500,40.043900,145,0.0,0.0,...,0.00000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000
3,chr1,180636,182055,3,6.703357,106618.940101,24609.011579,144,0.0,0.0,...,6.51873,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,4.68577
4,chr1,183251,183855,4,3.734692,20.225199,5.923574,67,0.0,0.0,...,4.93771,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000


In [9]:
# rename FLAG_MEF2D to MEF2D
bindings.rename(columns = {"FLAG_MEF2D": "MEF2D"}, inplace=True)

In [10]:
# getting the crcs
crcs = helper.fileToList("../data/CRCs_31.txt")

In [11]:
# open SE matrix
superenhancers = pd.read_excel("../data/superenhancer/"+se_matrix,
                               header=None, index_col=None)

In [13]:
# get only region and gene cols
superenhancers = superenhancers.iloc[:,[0,1,2,3,4,11]]

In [14]:
superenhancers.head()

,0,1,2,3,4,11
0,SE_N,REGION_ID,START.HG19,STOP.HG19,CHROM,ALL_GENES
1,1,1_merged_PDX_cell_lines_AML_1_lociStitched,11850225,12017830,chr12,APOLD1 CDKN1B CLEC4E CREBL2 ETV6 GPR19 MANSC1 ...
2,2,1_merged_PDX_cell_lines_AML_2_lociStitched,30753792,30869320,chr4,PCDH7
3,3,1_merged_PDX_cell_lines_AML_3_lociStitched,18375636,18489680,chr3,SATB1
4,4,1_merged_PDX_cell_lines_AML_4_lociStitched,134685054,134795163,chr5,C5orf20 H2AFY PITX1 SLC25A48 TIFAB


In [15]:
# set column names
superenhancers.columns = superenhancers.iloc[0]
superenhancers.drop(superenhancers.index[0], inplace=True)

In [16]:
# rename chrom info to match bindings
superenhancers.rename(columns = {"START.HG19": "start", "STOP.HG19": "end", "CHROM": "chrom"}, inplace=True)

In [17]:
superenhancers.head()

,SE_N,REGION_ID,start,end,chrom,ALL_GENES
1,1,1_merged_PDX_cell_lines_AML_1_lociStitched,11850225,12017830,chr12,APOLD1 CDKN1B CLEC4E CREBL2 ETV6 GPR19 MANSC1 ...
2,2,1_merged_PDX_cell_lines_AML_2_lociStitched,30753792,30869320,chr4,PCDH7
3,3,1_merged_PDX_cell_lines_AML_3_lociStitched,18375636,18489680,chr3,SATB1
4,4,1_merged_PDX_cell_lines_AML_4_lociStitched,134685054,134795163,chr5,C5orf20 H2AFY PITX1 SLC25A48 TIFAB
5,5,1_merged_PDX_cell_lines_AML_5_lociStitched,9863066,9968827,chr12,CD69 CLEC12A CLEC2D CLECL1 KLRF1 KLRF2 M6PR


In [18]:
bindings.head()

,chrom,start,end,peak_number,foldchange,-log10pvalue,-log10qvalue,relative_summit_pos,CEBPA,HOXA9,...,TFAP4,RUNX2,MAX,IRF8,MYB,ZEB2,RXRA,SPI1,MEF2C,MEF2D
0,chr1,9936,10300,0,10.264422,31242.919925,20942.926956,107,0.0,0.0,...,20.13070,5.750405,0.0,0.0,0.0,0.0,8.51315,0.0,0.0,3.86739
1,chr1,15524,15801,1,3.326400,9.112810,7.086980,277,0.0,0.0,...,0.00000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000
2,chr1,29182,29649,2,15.734000,43.992500,40.043900,145,0.0,0.0,...,0.00000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000
3,chr1,180636,182055,3,6.703357,106618.940101,24609.011579,144,0.0,0.0,...,6.51873,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,4.68577
4,chr1,183251,183855,4,3.734692,20.225199,5.923574,67,0.0,0.0,...,4.93771,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000


# Map cobinding matrix to SE regions

map cobinding peaks to SE regions

In [19]:
window = 150
merging_version = "simpleMerge"

In [20]:
superenhancers.shape

(6868, 6)

In [21]:
bindings.shape

(383704, 37)

In [24]:
# one region has chrom as 1 (int)
superenhancers.loc[superenhancers["chrom"] == 1,]

,SE_N,REGION_ID,start,end,chrom,ALL_GENES
1201,1243,1_merged_PDX_cell_lines_AML_1243_lociStitched,88518136,88536335,1,ZC3H18 ZFPM1 ZNF469


In [25]:
SEs = superenhancers.sort_values(by=["chrom", "start", "end"])
# replace 1 with chr1
SEs["chrom"] = SEs["chrom"].replace([1], "chr1")
# sort SE peaks by chr and start
SEs = SEs.sort_values(by=["chrom", "start", "end"])

In [30]:
# SE regions
conscensus = SEs.sort_values(by=["chrom", "start", "end"]).reset_index(drop=True)

In [32]:
# sort binding peaks by chr and start
peaks = bindings.sort_values(by=["chrom", "start", "end"]).reset_index(drop=True)
# name peak regions
peaks["peak_number"] = peaks["peak_number"].apply(lambda row: "cobinding_peak_"+str(row+1))
peaks.shape

(383704, 37)

In [33]:
# remove weird chromosomes
peaks = peaks[~peaks["chrom"].str.contains("_")]
# set coords to int
peaks.start = peaks.start.astype(int)
peaks.end = peaks.end.astype(int)
peaks.shape

(382746, 37)

In [36]:
# get only peaks w/ same chromosomes as conscensus
peaks = peaks[peaks["chrom"].isin(conscensus["chrom"])]
peaks.shape

(382027, 37)

__overlapping testing__

In [163]:
mergedpeaksdict = {}
peaknumber=0
prev_start=conscensus.iloc[0]["start"]
prev_end=conscensus.iloc[0]["end"]
prev_chrom=conscensus.iloc[0]["chrom"]

In [211]:
mergedpeaksdict={}
for idx, row in conscensus.iloc[:2].iterrows():
    peak_dict = {}
    value = peaks.loc[peaks["chrom"] == row.chrom]
    print(values.shape)

    for i, b in value.iloc[:5].iterrows():
        if b.start<row.start:
            print(b.index[8:][b.iloc[8:] != 0].tolist())
            peak_dict[b.peak_number] = b.index[8:][b.iloc[8:] != 0].tolist()
    mergedpeaksdict[row.REGION_ID] = peak_dict

(31340, 37)
['GFI1', 'RUNX1', 'SP1', 'E2F3', 'STAT5B', 'TFAP4', 'RUNX2', 'RXRA', 'MEF2D']
['MYC']
['PLAGL2']
['MEIS1', 'GFI1', 'RUNX1', 'SP1', 'ZNF281', 'STAT5B', 'MYC', 'TFAP4', 'MEF2D']
['ZNF281', 'MYC', 'TFAP4']
(31340, 37)
['GFI1', 'RUNX1', 'SP1', 'E2F3', 'STAT5B', 'TFAP4', 'RUNX2', 'RXRA', 'MEF2D']
['MYC']
['PLAGL2']
['MEIS1', 'GFI1', 'RUNX1', 'SP1', 'ZNF281', 'STAT5B', 'MYC', 'TFAP4', 'MEF2D']
['ZNF281', 'MYC', 'TFAP4']


In [212]:
mergedpeaksdict

{'1_merged_PDX_cell_lines_AML_7731_lociStitched': {'cobinding_peak_1': ['GFI1',
   'RUNX1',
   'SP1',
   'E2F3',
   'STAT5B',
   'TFAP4',
   'RUNX2',
   'RXRA',
   'MEF2D'],
  'cobinding_peak_2': ['MYC'],
  'cobinding_peak_3': ['PLAGL2'],
  'cobinding_peak_4': ['MEIS1',
   'GFI1',
   'RUNX1',
   'SP1',
   'ZNF281',
   'STAT5B',
   'MYC',
   'TFAP4',
   'MEF2D'],
  'cobinding_peak_5': ['ZNF281', 'MYC', 'TFAP4']},
 '1_merged_PDX_cell_lines_AML_1367_lociStitched': {'cobinding_peak_1': ['GFI1',
   'RUNX1',
   'SP1',
   'E2F3',
   'STAT5B',
   'TFAP4',
   'RUNX2',
   'RXRA',
   'MEF2D'],
  'cobinding_peak_2': ['MYC'],
  'cobinding_peak_3': ['PLAGL2'],
  'cobinding_peak_4': ['MEIS1',
   'GFI1',
   'RUNX1',
   'SP1',
   'ZNF281',
   'STAT5B',
   'MYC',
   'TFAP4',
   'MEF2D'],
  'cobinding_peak_5': ['ZNF281', 'MYC', 'TFAP4']}}

In [218]:
pd.DataFrame.from_dict(mergedpeaksdict)

,1_merged_PDX_cell_lines_AML_7731_lociStitched,1_merged_PDX_cell_lines_AML_1367_lociStitched
cobinding_peak_1,"[GFI1, RUNX1, SP1, E2F3, STAT5B, TFAP4, RUNX2,...","[GFI1, RUNX1, SP1, E2F3, STAT5B, TFAP4, RUNX2,..."
cobinding_peak_2,[MYC],[MYC]
cobinding_peak_3,[PLAGL2],[PLAGL2]
cobinding_peak_4,"[MEIS1, GFI1, RUNX1, SP1, ZNF281, STAT5B, MYC,...","[MEIS1, GFI1, RUNX1, SP1, ZNF281, STAT5B, MYC,..."
cobinding_peak_5,"[ZNF281, MYC, TFAP4]","[ZNF281, MYC, TFAP4]"


In [ ]:
mergedpeaksdict={}
for idx, row in conscensus.iloc[:2].iterrows():
    peak_dict = {}
    value = peaks.loc[peaks["chrom"] == row.chrom]
    print(values.shape)

    for i, b in value.iloc[:5].iterrows():
        if b.start<row.start:
            print(b.index[8:][b.iloc[8:] != 0].tolist())
            peak_dict[b.peak_number] = b.index[8:][b.iloc[8:] != 0].tolist()
    mergedpeaksdict[row.REGION_ID] = peak_dict

In [224]:
conscensus.head()

,SE_N,REGION_ID,start,end,chrom,ALL_GENES
0,7731,1_merged_PDX_cell_lines_AML_7731_lociStitched,1076094,1083036,chr1,C1orf159 MIR200A MIR200B MIR429 TTLL10
1,1367,1_merged_PDX_cell_lines_AML_1367_lociStitched,1138179,1155728,chr1,B3GALT6 FAM132A MIR200A MIR200B MIR429 SDF4 SM...
2,8547,1_merged_PDX_cell_lines_AML_8547_lociStitched,1365724,1372115,chr1,ATAD3A ATAD3B ATAD3C AURKAIP1 CCNL2 DVL1 LOC14...
3,16159,1_merged_PDX_cell_lines_AML_16159_lociStitched,1589785,1591611,chr1,ATAD3A CDK11B MIB2 MMP23A MMP23B MORN1 RER1 SL...
4,4799,1_merged_PDX_cell_lines_AML_4799_lociStitched,1706094,1715621,chr1,GABRD GNB1 NADK SDF4 SLC35E2


__overlapping conditions__
* peak overlaps SE start (peak.start < SE.start and peak.end > SE.start)
* peak overlaps SE end (peak.start > SE.start and peak.end > SE.end)
* peak within SE (peak.start > SE.start and peak.end < SE.end)
* peak larger than SE (peak.start < SE.start and peak.end > SE.end)

In [330]:
# presence df
res_cols = conscensus.columns.tolist()+["N_PEAKS"]+peaks.columns.tolist()[8:]
res_rows = []

# all peaks df columns
split_cols = conscensus.columns.tolist()+[peaks.columns.tolist()[3]]+peaks.columns.tolist()[8:]
# rows of df with all peaks
split_rows = []

for idx, row in conscensus.iloc[:2].iterrows():
    # get only peaks w/ same chromosome
    peaks_chr = peaks.loc[peaks["chrom"] == row.chrom]
    # make list of 0s for each tf +1 for # peaks
    merged_res = np.zeros(len(mergedres_cols[6:]))
    # for each peak
    for i, peak in peaks_chr.iloc[80:90].iterrows():
        # peak overlaps SE start or peak is larger than SE
        # peak on or greater than start or peak within
        if (peak.start<row.start and peak.end>row.start) \
        or (peak.start>=row.start and peak.start<row.end):
            merged_res = np.add(merged_res, [1]+[1 if x!= 0 else 0 for x in peak[8:].tolist()])
            split_rows.append(row.tolist()+[peak.peak_number]+peak[8:].tolist())
    res_rows.append(row.tolist()+merged_res.tolist())

In [331]:
# SEs with merged peaks
res = pd.DataFrame(res_rows, columns=res_cols)
print(res.shape)
# SEs per merged peak
res_split = pd.DataFrame(split_rows, columns=split_cols)
print(res_split.shape)

(2, 36)
(1, 36)


In [332]:
res

,SE_N,REGION_ID,start,end,chrom,ALL_GENES,N_PEAKS,CEBPA,HOXA9,PLAGL2,...,TFAP4,RUNX2,MAX,IRF8,MYB,ZEB2,RXRA,SPI1,MEF2C,MEF2D
0,7731,1_merged_PDX_cell_lines_AML_7731_lociStitched,1076094,1083036,chr1,C1orf159 MIR200A MIR200B MIR429 TTLL10,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1367,1_merged_PDX_cell_lines_AML_1367_lociStitched,1138179,1155728,chr1,B3GALT6 FAM132A MIR200A MIR200B MIR429 SDF4 SM...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [333]:
res_split

,SE_N,REGION_ID,start,end,chrom,ALL_GENES,peak_number,CEBPA,HOXA9,PLAGL2,...,TFAP4,RUNX2,MAX,IRF8,MYB,ZEB2,RXRA,SPI1,MEF2C,MEF2D
0,7731,1_merged_PDX_cell_lines_AML_7731_lociStitched,1076094,1083036,chr1,C1orf159 MIR200A MIR200B MIR429 TTLL10,cobinding_peak_84,0.0,0.0,0.0,...,0.0,0.0,8.739785,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [286]:
# SE = [4, 8]
for x,y in [[2,5], [5,7], [7,9], [2, 10], [0,3], [9,12]]:
    print(x, y)
    if (x<=4 and y>4) or (x>=4 and x<8):
        print("overlapping")
    else:
        print("not overlapping")

2 5
overlapping
5 7
overlapping
7 9
overlapping
2 10
overlapping
0 3
not overlapping
9 12
not overlapping


In [292]:
print(conscensus.iloc[0].start, conscensus.iloc[0].end)

1076094 1083036


In [311]:
["yes" if (x.start<conscensus.iloc[0].start and x.end>conscensus.iloc[0].start) or
 (x.start>=conscensus.iloc[0].start and x.start<conscensus.iloc[0].end) else "no" 
 for i,x in peaks_chr.iloc[80:90].iterrows()]

['no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no']

In [310]:
print(conscensus.iloc[0].start, conscensus.iloc[0].end)
print("\n")

for i,x in peaks_chr.iloc[80:90].iterrows():
    print(x.start, x.end)
    if (x.start < conscensus.iloc[0].start and x.end > conscensus.iloc[0].start):
        print("overlapping start")
        
    elif (x.start >= conscensus.iloc[0].start and x.start <= conscensus.iloc[0].end):
        print("overlapping end or within")
    
    else:
        print("not overlapping")

1076094 1083036


1067690 1068155
not overlapping
1068551 1069388
not overlapping
1074770 1075098
not overlapping
1079409 1080385
overlapping end or within
1091300 1091850
not overlapping
1097970 1098314
not overlapping
1103921 1104665
not overlapping
1112303 1112734
not overlapping
1115570 1116445
not overlapping
1117255 1117850
not overlapping


__run over both matricies__

In [37]:
print("SE regions: {}".format(conscensus.shape[0]))
print("Binding peaks: {}".format(peaks.shape[0]))

SE regions: 6868
Binding peaks: 382027


In [336]:
print("SE regions: {}".format(conscensus.shape[0]))
print("Binding peaks: {}".format(peaks.shape[0]))

SE regions: 6868
Binding peaks: 382027


In [41]:
len(conscensus.columns.tolist()+["N_PEAKS"]+peaks.columns.tolist()[8:])

36

In [42]:
len(conscensus.columns.tolist()+[peaks.columns.tolist()[3]]+peaks.columns.tolist()[8:])

36

In [ ]:
%%time

# presence df
res_cols = conscensus.columns.tolist()+["N_PEAKS"]+peaks.columns.tolist()[8:]
res_rows = []

# all peaks df columns
split_cols = conscensus.columns.tolist()+[peaks.columns.tolist()[3]]+peaks.columns.tolist()[8:]
# rows of df with all peaks
split_rows = []

ix = 0
for idx, row in conscensus.iterrows():
    # get only peaks w/ same chromosome
    peaks_chr = peaks.loc[peaks["chrom"] == row.chrom]
    # make list of 0s for each tf +1 for # peaks
    merged_res = np.zeros(len(res_cols[6:]))
    # for each peak
    i = 0
    for i, peak in peaks_chr.iterrows():
        # peak overlaps SE start or peak is larger than SE
        # peak on or greater than start or peak within
        if (peak.start<row.start and peak.end>row.start) \
        or (peak.start>=row.start and peak.start<row.end):
            merged_res = np.add(merged_res, [1]+[1 if x!= 0 else 0 for x in peak[8:].tolist()])
            split_rows.append(row.tolist()+[peak.peak_number]+peak[8:].tolist())
            i = i+1
        else:
            i = i+1
    #print("Binding peaks mapped: {}".format(row.REGION_ID))
    res_rows.append(row.tolist()+merged_res.tolist())
    ix = ix+1
    if (ix%1000 == 0): print("{} SEs mapped to".format(ix))


# SEs with merged peaks
res = pd.DataFrame(res_rows, columns=res_cols)
print(res.shape)
# SEs per merged peak
res_split = pd.DataFrame(split_rows, columns=split_cols)
print(res_split.shape)

1000 SEs mapped to
2000 SEs mapped to
3000 SEs mapped to


In [356]:
%%time

# presence df
res_cols = conscensus.columns.tolist()+["N_PEAKS"]+peaks.columns.tolist()[8:]
res_rows = []

# all peaks df columns
split_cols = conscensus.columns.tolist()+[peaks.columns.tolist()[3]]+peaks.columns.tolist()[8:]
# rows of df with all peaks
split_rows = []

ix = 0
for idx, row in conscensus.iloc[1000:].iterrows():
    # get only peaks w/ same chromosome
    peaks_chr = peaks.loc[peaks["chrom"] == row.chrom]
    # make list of 0s for each tf +1 for # peaks
    merged_res = np.zeros(len(mergedres_cols[6:]))
    # for each peak
    i = 0
    for i, peak in peaks_chr.iterrows():
        # peak overlaps SE start or peak is larger than SE
        # peak on or greater than start or peak within
        if (peak.start<row.start and peak.end>row.start) \
        or (peak.start>=row.start and peak.start<row.end):
            merged_res = np.add(merged_res, [1]+[1 if x!= 0 else 0 for x in peak[8:].tolist()])
            split_rows.append(row.tolist()+[peak.peak_number]+peak[8:].tolist())
            i = i+1
        else:
            i = i+1
    #print("Binding peaks mapped: {}".format(row.REGION_ID))
    res_rows.append(row.tolist()+merged_res.tolist())
    ix = ix+1
    if (ix%1000 == 0): print("{} SEs mapped to".format(ix))


# SEs with merged peaks
res = pd.DataFrame(res_rows, columns=res_cols)
print(res.shape)
# SEs per merged peak
res_split = pd.DataFrame(split_rows, columns=split_cols)
print(res_split.shape)

1000 SEs mapped to
2000 SEs mapped to
3000 SEs mapped to
4000 SEs mapped to
5000 SEs mapped to
(5868, 36)
(13701, 36)
CPU times: user 1h 48min 22s, sys: 312 ms, total: 1h 48min 23s
Wall time: 1h 48min 23s


In [350]:
# first 1000 SE regions
res_base = res
res_split_base = res_split

In [351]:
print(res_base.shape)
print(res_split_base.shape)

(1000, 36)
(2090, 36)


In [359]:
# last 5868 SE regions
res_new = res
res_split_new = res_split

In [364]:
# combine all SE regions
res = pd.concat([res_base, res_new])
res_split = pd.concat([res_split_base, res_split_new])

In [365]:
print(res.shape)
print(res_split.shape)

(6868, 36)
(15791, 36)


In [371]:
# save to csv
res.to_csv("../results/"+cobinding_project+"/"+cobinding_version+"/"+cobinding_version+
           "_SE_matrix_merged.csv.gz", index=False)
res_split.to_csv("../results/"+cobinding_project+"/"+cobinding_version+"/"+cobinding_version+
                 "_SE_matrix_merged_split.csv.gz", index=False)

## Map genes to SE

* res = cobinding mapped to SE regions
* SEs = sorted SE matrix

In [375]:
SEs = SEs.sort_values(by=["chrom", "start", "end"]).reset_index(drop=True)

In [378]:
SEs.head()

,SE_N,REGION_ID,start,end,chrom,ALL_GENES
0,7731,1_merged_PDX_cell_lines_AML_7731_lociStitched,1076094,1083036,chr1,C1orf159 MIR200A MIR200B MIR429 TTLL10
1,1367,1_merged_PDX_cell_lines_AML_1367_lociStitched,1138179,1155728,chr1,B3GALT6 FAM132A MIR200A MIR200B MIR429 SDF4 SM...
2,8547,1_merged_PDX_cell_lines_AML_8547_lociStitched,1365724,1372115,chr1,ATAD3A ATAD3B ATAD3C AURKAIP1 CCNL2 DVL1 LOC14...
3,16159,1_merged_PDX_cell_lines_AML_16159_lociStitched,1589785,1591611,chr1,ATAD3A CDK11B MIB2 MMP23A MMP23B MORN1 RER1 SL...
4,4799,1_merged_PDX_cell_lines_AML_4799_lociStitched,1706094,1715621,chr1,GABRD GNB1 NADK SDF4 SLC35E2


seperate all genes and get list of unique genes

In [382]:
SEs['ALL_GENES'].astype(str).apply(lambda x: x.split(" "))

0            [C1orf159, MIR200A, MIR200B, MIR429, TTLL10]
1       [B3GALT6, FAM132A, MIR200A, MIR200B, MIR429, S...
2       [ATAD3A, ATAD3B, ATAD3C, AURKAIP1, CCNL2, DVL1...
3       [ATAD3A, CDK11B, MIB2, MMP23A, MMP23B, MORN1, ...
4                      [GABRD, GNB1, NADK, SDF4, SLC35E2]
                              ...                        
6863    [ARHGAP4, HCFC1, IRAK1, MIR3202-1, MIR3202-2, ...
6864           [DNASE1L1, EMD, FLNA, RPL10, SNORA70, TAZ]
6865                      [DKC1, GAB3, SNORA36A, SNORA56]
6866    [DKC1, GAB3, LOC100132963, MPP1, SNORA36A, SNO...
6867                                               [IL9R]
Name: ALL_GENES, Length: 6868, dtype: object

In [399]:
%%time

unique_genes = []
for idx, row in SEs.iloc[:20].iterrows():
    region = row["REGION_ID"]
    gene_list = row["ALL_GENES"]
    for gene in gene_list.split(" "):
        if gene not in unique_genes:
            unique_genes.append(gene)

print(len(unique_genes))

71
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.23 ms


In [403]:
sorted(unique_genes)

['ACAP3',
 'ACTRT2',
 'ARHGEF16',
 'ATAD3A',
 'ATAD3B',
 'ATAD3C',
 'AURKAIP1',
 'B3GALT6',
 'C1orf159',
 'C1orf174',
 'C1orf86',
 'C1orf93',
 'CALML6',
 'CCDC27',
 'CCNL2',
 'CDK11B',
 'CEP104',
 'DFFB',
 'DVL1',
 'FAM132A',
 'FLJ42875',
 'GABRD',
 'GNB1',
 'HES5',
 'KIAA0495',
 'LOC100128003',
 'LOC100129534',
 'LOC115110',
 'LOC148413',
 'LOC441869',
 'LRRC47',
 'MEGF6',
 'MIB2',
 'MIR200A',
 'MIR200B',
 'MIR4251',
 'MIR429',
 'MIR551A',
 'MMEL1',
 'MMP23A',
 'MMP23B',
 'MORN1',
 'MRPL20',
 'NADK',
 'PANK4',
 'PEX10',
 'PRDM16',
 'PRKCZ',
 'PUSL1',
 'RER1',
 'SAMD11',
 'SDF4',
 'SKI',
 'SLC35E2',
 'SLC35E2B',
 'SMIM1',
 'SSU72',
 'THAP3',
 'TMEM52',
 'TMEM88B',
 'TNFRSF14',
 'TNFRSF18',
 'TNFRSF25',
 'TNFRSF4',
 'TP73',
 'TPRG1L',
 'TTLL10',
 'UBE2J2',
 'VWA1',
 'WDR8',
 'WRAP73']

In [406]:
res.iloc[:1]

,SE_N,REGION_ID,start,end,chrom,ALL_GENES,N_PEAKS,CEBPA,HOXA9,PLAGL2,...,TFAP4,RUNX2,MAX,IRF8,MYB,ZEB2,RXRA,SPI1,MEF2C,MEF2D
0,7731,1_merged_PDX_cell_lines_AML_7731_lociStitched,1076094,1083036,chr1,C1orf159 MIR200A MIR200B MIR429 TTLL10,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [408]:
crcs

['CEBPA',
 'E2F3',
 'FLI1',
 'FOSL2',
 'GFI1',
 'HHEX',
 'IRF8',
 'LYL1',
 'MEF2C',
 'MEF2D',
 'MEIS1',
 'MYB',
 'PLAGL2',
 'RUNX1',
 'RUNX2',
 'RXRA',
 'SP1',
 'SPI1',
 'SREBF1',
 'STAT5B',
 'TFAP4',
 'ZEB2',
 'ZFPM1',
 'ZNF281',
 'GATA2',
 'MAX',
 'MYC',
 'ETV6',
 'HOXA9',
 'LMO2',
 'ZMYND']

In [423]:
bindings.columns.tolist()[-29:]

['CEBPA',
 'HOXA9',
 'PLAGL2',
 'MEIS1',
 'ETV6',
 'GFI1',
 'RUNX1',
 'FOSL2',
 'FLI1',
 'GATA2',
 'SP1',
 'E2F3',
 'ZNF281',
 'SREBF1',
 'LMO2',
 'STAT5B',
 'ZMYND8',
 'LYL1',
 'MYC',
 'TFAP4',
 'RUNX2',
 'MAX',
 'IRF8',
 'MYB',
 'ZEB2',
 'RXRA',
 'SPI1',
 'MEF2C',
 'MEF2D']

In [459]:
res.iloc[:1]

,SE_N,REGION_ID,start,end,chrom,ALL_GENES,N_PEAKS,CEBPA,HOXA9,PLAGL2,...,TFAP4,RUNX2,MAX,IRF8,MYB,ZEB2,RXRA,SPI1,MEF2C,MEF2D
0,7731,1_merged_PDX_cell_lines_AML_7731_lociStitched,1076094,1083036,chr1,C1orf159 MIR200A MIR200B MIR429 TTLL10,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [460]:
res.columns.tolist()[6:]

['N_PEAKS',
 'CEBPA',
 'HOXA9',
 'PLAGL2',
 'MEIS1',
 'ETV6',
 'GFI1',
 'RUNX1',
 'FOSL2',
 'FLI1',
 'GATA2',
 'SP1',
 'E2F3',
 'ZNF281',
 'SREBF1',
 'LMO2',
 'STAT5B',
 'ZMYND8',
 'LYL1',
 'MYC',
 'TFAP4',
 'RUNX2',
 'MAX',
 'IRF8',
 'MYB',
 'ZEB2',
 'RXRA',
 'SPI1',
 'MEF2C',
 'MEF2D']

In [470]:
%%time

# presence df
df_cols = res.columns.tolist()[-30:] #["GENE", "CRC"]
res_dict = {}
unique_genes = []

ix = 0
for idx, row in res.iloc[:1000].iterrows():
    
    # make list of 0s for each tf +1 for # peaks
    #merged_res = np.zeros(len(mergedres_cols[-29:]))
    
    region = row["REGION_ID"]
    gene_list = str(row["ALL_GENES"]).split(" ")
    for gene in gene_list:
        if gene not in unique_genes:
            unique_genes.append(gene)
        if gene in res_dict:
            res_dict[gene] = np.add(res_dict[gene], row[6:].tolist())
        else:
            res_dict[gene] = row[6:].tolist()

    ix = ix+1
    if (ix%1000 == 0): print("{} SEs processed".format(ix))


# SE genes with peaks
df = pd.DataFrame.from_dict(res_dict, orient="index", columns=df_cols)
print("Total genes: {}".format(len(unique_genes)))
print("Genes in df: {}".format(df.shape[0]))

1000 SEs processed
Total genes: 1707
Genes in df: 1707
CPU times: user 196 ms, sys: 0 ns, total: 196 ms
Wall time: 194 ms


Time measures - SEs (genes)
* 10 (52) - CPU: 4ms / Wall: 5.46ms
* 100 (235) - CPU: 20ms / Wall: 21.5ms
* 500 (1072) - CPU: 112ms / Wall: 111ms
* 1000 (1707) - CPU: 196ms / Wall: 194ms

In [471]:
df.head()

,N_PEAKS,CEBPA,HOXA9,PLAGL2,MEIS1,ETV6,GFI1,RUNX1,FOSL2,FLI1,...,TFAP4,RUNX2,MAX,IRF8,MYB,ZEB2,RXRA,SPI1,MEF2C,MEF2D
C1orf159,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MIR200A,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,...,0.0,0.0,3.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
MIR200B,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,...,0.0,0.0,3.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
MIR429,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,...,0.0,0.0,3.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
TTLL10,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,...,0.0,0.0,3.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0


In [486]:
superenhancers.loc[superenhancers['ALL_GENES'].str.contains("CEBPA", case=False)]

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [493]:
superenhancers.iloc[39]

0
SE_N                                                        40
REGION_ID          1_merged_PDX_cell_lines_AML_40_lociStitched
start                                                 33739117
end                                                   33795715
chrom                                                    chr19
ALL_GENES    CCNE1 CEBPA GRAMD1A HSPB6 KCTD15 LOC80054 LRP3...
Name: 40, dtype: object

In [490]:
superenhancers['ALL_GENES'].str.contains("CEBPA", case=False)[superenhancers['ALL_GENES'].str.contains("CEBPA", case=False) != False]

40      True
1426     NaN
2087     NaN
2503     NaN
3121     NaN
4068     NaN
5187     NaN
5938     NaN
6394     NaN
Name: ALL_GENES, dtype: object

In [499]:
superenhancers.iloc[[39,1425,2086,2502,3120,4067,5186,5937,6393],:]

,SE_N,REGION_ID,start,end,chrom,ALL_GENES
40,40,1_merged_PDX_cell_lines_AML_40_lociStitched,33739117,33795715,chr19,CCNE1 CEBPA GRAMD1A HSPB6 KCTD15 LOC80054 LRP3...
1426,1497,1_merged_PDX_cell_lines_AML_1497_lociStitched,45960034,45976885,chr10,43898
2087,2264,1_merged_PDX_cell_lines_AML_2264_lociStitched,164508601,164522551,chr4,43891
2503,2818,1_merged_PDX_cell_lines_AML_2818_lociStitched,75560044,75572674,chr17,44083
3121,3663,1_merged_PDX_cell_lines_AML_3663_lociStitched,77906409,77917457,chr4,44085
4068,5156,1_merged_PDX_cell_lines_AML_5156_lociStitched,164527064,164536207,chr4,43891
5187,7529,1_merged_PDX_cell_lines_AML_7529_lociStitched,164582088,164589162,chr4,43891
5938,9659,1_merged_PDX_cell_lines_AML_9659_lociStitched,77926134,77931811,chr4,44085
6394,11386,1_merged_PDX_cell_lines_AML_11386_lociStitched,164476769,164481461,chr4,43891


In [483]:
print(len(crcs))
crcs

31


['CEBPA',
 'E2F3',
 'FLI1',
 'FOSL2',
 'GFI1',
 'HHEX',
 'IRF8',
 'LYL1',
 'MEF2C',
 'MEF2D',
 'MEIS1',
 'MYB',
 'PLAGL2',
 'RUNX1',
 'RUNX2',
 'RXRA',
 'SP1',
 'SPI1',
 'SREBF1',
 'STAT5B',
 'TFAP4',
 'ZEB2',
 'ZFPM1',
 'ZNF281',
 'GATA2',
 'MAX',
 'MYC',
 'ETV6',
 'HOXA9',
 'LMO2',
 'ZMYND']

In [496]:
df.head()

,N_PEAKS,CEBPA,HOXA9,PLAGL2,MEIS1,ETV6,GFI1,RUNX1,FOSL2,FLI1,...,TFAP4,RUNX2,MAX,IRF8,MYB,ZEB2,RXRA,SPI1,MEF2C,MEF2D
C1orf159,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MIR200A,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,...,0.0,0.0,3.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
MIR200B,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,...,0.0,0.0,3.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
MIR429,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,...,0.0,0.0,3.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
TTLL10,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,...,0.0,0.0,3.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0


In [498]:
df.loc["MIR200A"]

N_PEAKS    3.0
CEBPA      0.0
HOXA9      0.0
PLAGL2     0.0
MEIS1      0.0
ETV6       0.0
GFI1       0.0
RUNX1      3.0
FOSL2      0.0
FLI1       1.0
GATA2      0.0
SP1        1.0
E2F3       0.0
ZNF281     1.0
SREBF1     0.0
LMO2       1.0
STAT5B     0.0
ZMYND8     1.0
LYL1       0.0
MYC        2.0
TFAP4      0.0
RUNX2      0.0
MAX        3.0
IRF8       0.0
MYB        1.0
ZEB2       1.0
RXRA       1.0
SPI1       0.0
MEF2C      0.0
MEF2D      0.0
Name: MIR200A, dtype: float64

In [476]:
crc_genes = pd.Series([1 if gene not in crcs else 0 for gene in df.index.tolist()], index=df.index.tolist())

In [484]:
#[1 if x not in crcs else 0 for x in df.index.tolist()]
[x for x in sorted(df.index.tolist())]

['43898',
 'ABCA4',
 'ABCB10',
 'ABCD3',
 'ABI1',
 'ABL2',
 'ACAP3',
 'ACBD3',
 'ACBD5',
 'ACBD7',
 'ACOT11',
 'ACOT7',
 'ACTA2',
 'ACTR1A',
 'ACTRT2',
 'ADAM15',
 'ADAMTS14',
 'ADAMTS4',
 'ADAMTSL4',
 'ADAR',
 'ADARB2',
 'ADC',
 'ADCK3',
 'ADCY10',
 'ADD3',
 'ADIPOR1',
 'ADK',
 'ADORA1',
 'ADORA3',
 'ADPRHL2',
 'ADRA2A',
 'AFAP1L2',
 'AGAP4',
 'AGAP7',
 'AGMAT',
 'AGO1',
 'AGO3',
 'AGTRAP',
 'AHCYL1',
 'AHDC1',
 'AK2',
 'AK4',
 'AK5',
 'AKNAD1',
 'AKR1A1',
 'AKR1C1',
 'AKR1E2',
 'AKR7A2',
 'AKR7A3',
 'AKR7L',
 'AKT3',
 'ALDH18A1',
 'ALDH4A1',
 'ALDH9A1',
 'ALG14',
 'ALOX5',
 'AMIGO1',
 'AMPD2',
 'ANAPC16',
 'ANGPTL1',
 'ANKRD1',
 'ANKRD16',
 'ANKRD22',
 'ANKRD26',
 'ANKRD34A',
 'ANKRD36BP1',
 'ANP32E',
 'ANXA2P3',
 'ANXA7',
 'ANXA9',
 'AP4B1',
 'APBB1IP',
 'APH1A',
 'APITD1',
 'APITD1-CORT',
 'APOA2',
 'APOBEC4',
 'AQP10',
 'ARF1',
 'ARHGAP19',
 'ARHGAP22',
 'ARHGAP30',
 'ARHGEF10L',
 'ARHGEF11',
 'ARHGEF16',
 'ARHGEF2',
 'ARID1A',
 'ARID4B',
 'ARID5B',
 'ARL3',
 'ARL5B',
 'ARL8A',
 '

In [456]:
mydict = {}
for idx, row in test.iterrows():
    region = row["SE"]
    genes = row["GENE"]
    #counts = row[2:].tolist()
    for gene in genes:
        if gene in mydict:
            #print(row[2:])
            mydict[gene] = np.add(mydict[gene], row[2:].tolist())
        else:
            mydict[gene] = row[2:].tolist()

pd.DataFrame.from_dict(mydict, orient="index", columns=test.columns.tolist()[2:])

,MAX,MEF2C,MEF2D,MEIS1,MYB
GENE1,1,2,2,2,3
GENE2,0,0,1,1,1
GENE3,1,1,2,1,2
GENE4,1,1,0,2,1
GENE5,2,2,1,2,2
GENE6,1,0,0,1,0


In [452]:
data = [["SE1",["GENE1", "GENE2", "GENE3"],0,0,1,0,1],
        ["SE2",["GENE4", "GENE5", "GENE6"],1,0,0,0,0],
        ["SE3",["GENE1", "GENE3", "GENE5"],1,1,1,1,1],
        ["SE4",["GENE2", "GENE4", "GENE6"],0,0,0,1,0],
        ["SE5",["GENE1", "GENE4", "GENE5"],0,1,0,1,1]]
test = pd.DataFrame(data, columns=["SE","GENE","MAX","MEF2C", "MEF2D","MEIS1","MYB"])
test

,SE,GENE,MAX,MEF2C,MEF2D,MEIS1,MYB
0,SE1,"[GENE1, GENE2, GENE3]",0,0,1,0,1
1,SE2,"[GENE4, GENE5, GENE6]",1,0,0,0,0
2,SE3,"[GENE1, GENE3, GENE5]",1,1,1,1,1
3,SE4,"[GENE2, GENE4, GENE6]",0,0,0,1,0
4,SE5,"[GENE1, GENE4, GENE5]",0,1,0,1,1


In [451]:
pd.DataFrame.from_dict({"GENE1":[1,2,2,2,3], "GENE2":[0,0,1,1,1], "GENE3":[1,1,2,1,2], 
                        "GENE4":[1,1,0,2,1], "GENE5":[2,2,1,2,2], "GENE6":[1,0,0,1,0]},
                        orient="index", columns=test.columns.tolist()[2:])

,MAX,MEF2C,MEF2D,MEIS1,MYB
GENE1,1,2,2,2,3
GENE2,0,0,1,1,1
GENE3,1,1,2,1,2
GENE4,1,1,0,2,1
GENE5,2,2,1,2,2
GENE6,1,0,0,1,0


# check Jost's SE matrix for genes

Issues with genes in certain regions. They do not have any assigned genes. It changed to a number. Might be an xlsx issue.

In [501]:
jk = pd.read_csv("/home/monika/AMLproject/data/superenhancer/superenhancer_matrix_jk.csv")

/home/monika/anaconda3/envs/slamseq/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [502]:
jk

,Unnamed: 0,CHROM,START.HG19,STOP.HG19,START.HG38,STOP.HG38,PROXIMAL_GENES,OVERLAP_GENES,CLOSEST_GENE,ABC_assignment ([Corr] >0.3 in cell lines),...,EOL1.4,Pearson Dep/SE,MLL-nonMLL Dep,"p, MLL-nonMLL Dep",Average Dep score,Average SE score cell lines,Average SE score primary,Correlation to MEF2D (pAML),Correlation to IRF8,Correlation to BCL2 dependency
0,0,chr1,188208220,188212962,188239089.0,188243831.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5,2,"0,0379143000748641","0,0920239347507136","-0,87233174050688"
1,1,chr14,31595175,31600330,31125969.0,31131124.0,NaN,HECTD1,HECTD1,NaN,...,NaN,NaN,NaN,NaN,NaN,2,3,"0,195477662590484","-0,144763956280788","-0,851385789021602"
2,2,chr2,120107154,120126341,119349578.0,119368765.0,DBI,DBI C2orf76,C2orf76,NaN,...,NaN,NaN,NaN,NaN,NaN,18,19,"0,671370252609654","0,408822184660836","-0,83625252406249"
3,3,chr11,14575144,14578998,14553598.0,14557452.0,PSMA1,PSMA1,PSMA1,PDE3B,...,"-2,20256386635","-0,105259020156355","0,0340497751521212","0,661128178413146","-2,07385252593529",3,3,"0,203617619318475","-0,0106547504639564","-0,832003040497709"
4,4,chr11,73169511,73175223,73458466.0,73464178.0,NaN,FAM168A,RELT,PAAF1,...,NaN,NaN,NaN,NaN,NaN,6,7,"0,467617461610967","0,251036001503711","-0,825472555837567"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18139,18139,chrX,1619536,1627373,NaN,NaN,ASMTL,P2RY8,P2RY8,NaN,...,NaN,NaN,NaN,NaN,NaN,0,4,"-0,161001239494127","0,00635055497410523",NaN
18140,18140,chrX,1581299,1591684,NaN,NaN,ASMTL,P2RY8,ASMTL,NaN,...,NaN,NaN,NaN,NaN,NaN,0,7,"-0,189069300511941","-0,209916254940075",NaN
18141,18141,chrX,1392194,1394458,NaN,NaN,CSF2RA MIR3690,CSF2RA,CSF2RA,NaN,...,NaN,NaN,NaN,NaN,NaN,0,1,"-0,218221505891962","0,0480840763877521",NaN
18142,18142,chrX,1653122,1657699,NaN,NaN,NaN,P2RY8,P2RY8,NaN,...,NaN,NaN,NaN,NaN,NaN,0,8,"-0,268990863330352","-0,163071456921095",NaN
